<a href="https://colab.research.google.com/github/okechukwuchude/Automating-Medical-Coding/blob/main/Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
pip install stanza

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 970.4/970.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import os
import json
import stanza
import csv
import pandas as pd

# Build an English pipeline
stanza.download('en', package='mimic', processors={'ner': 'i2b2'}) # download English model
nlp = stanza.Pipeline('en', package='mimic', processors={'ner': 'i2b2'}) # initialize English neural pipeline

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading these customized packages for language: en (English)...
| Processor       | Package        |
------------------------------------
| tokenize        | mimic          |
| pos             | mimic_charlm   |
| lemma           | mimic_nocharlm |
| depparse        | mimic_charlm   |
| ner             | i2b2           |
| backward_charlm | mimic          |
| forward_charlm  | mimic          |
| pretrain        | mimic          |



INFO:stanza:Downloaded file to /root/stanza_resources/en/tokenize/mimic.pt


INFO:stanza:Downloaded file to /root/stanza_resources/en/pos/mimic_charlm.pt


INFO:stanza:Downloaded file to /root/stanza_resources/en/lemma/mimic_nocharlm.pt


INFO:stanza:Downloaded file to /root/stanza_resources/en/depparse/mimic_charlm.pt


INFO:stanza:Downloaded file to /root/stanza_resources/en/ner/i2b2.pt


INFO:stanza:Downloaded file to /root/stanza_resources/en/backward_charlm/mimic.pt


INFO:stanza:Downloaded file to /root/stanza_resources/en/forward_charlm/mimic.pt


INFO:stanza:Downloaded file to /root/stanza_resources/en/pretrain/mimic.pt
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor | Package        |
------------------------------
| tokenize  | mimic          |
| pos       | mimic_charlm   |
| lemma     | mimic_nocharlm |
| depparse  | mimic_charlm   |
| ner       | i2b2           |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [12]:
# Function to process text using Stanza
def process_text_with_stanza(text):
    doc = nlp(text)
    return doc

In [13]:
# Global variable to store all extracted texts
extracted_texts = []

In [14]:
def extract_info_from_json(json_file_path, hadm_id_set):
    global extracted_texts

    # Open the JSON file
    with open(json_file_path, 'r') as file:
        # Load JSON data from the file
        data = json.load(file)

        # Extract 'hadm_id' and 'comment' from the JSON data
        hadm_id = data.get('hadm_id', None)
        comment = data.get('comment', None)

        # If 'hadm_id' is not found, print a warning message
        if hadm_id is None:
            print(f"Warning: 'hadm_id' not found in {json_file_path}")
            return None

        # Add 'hadm_id' to the set
        hadm_id_set.add(hadm_id)

        # Extract 'notes' from the JSON data
        notes = data.get('notes', [])

        # Iterate through each note
        for note in notes:
            note_info = {}  # Initialize a dictionary to store note information

            # Add 'hadm_id' to the note information
            note_info['hadm_id'] = hadm_id

            # Extract information from the note
            note_info['note_id'] = note.get('note_id', None)
            note_info['category'] = note.get('category', None)
            note_info['description'] = note.get('description', None)

            # Extract annotations from the note
            annotations = note.get('annotations', [])
            annotations_info = []  # Initialize a list to store annotation information

            # Iterate through each annotation in the note
            for annotation in annotations:
                annotation_info = {}  # Initialize a dictionary to store annotation information

                # Extract information from the annotation
                annotation_info['begin'] = annotation.get('begin', None)
                annotation_info['end'] = annotation.get('end', None)
                annotation_info['code'] = annotation.get('code', None)
                annotation_info['code_system'] = annotation.get('code_system', None)
                annotation_info['description'] = annotation.get('description', None)
                annotation_info['type'] = annotation.get('type', None)
                annotation_info['covered_text'] = annotation.get('covered_text', None)

                annotations_info.append(annotation_info)  # Append annotation information to the list

            note_info['annotations'] = annotations_info  # Add annotations information to the note
            note_info['text'] = note.get('text', None)

            # Process text with Stanza
            if note_info['text']:
                processed_text = process_text_with_stanza(note_info['text'])
                note_info['processed_text'] = processed_text

                # Extract sentence information and generate labels
                sentence_info = []
                for sent in processed_text.sentences:
                    tokens = [word.text for word in sent.words]
                    labels = ['O'] * len(tokens)  # Initialize labels as 'O'

                    for annotation in note_info['annotations']:
                        begin = annotation['begin']
                        end = annotation['end']

                        word_indices = find_word_indices(sent.words, begin, end)

                        # Assign labels
                        if word_indices:
                            start_idx, end_idx = word_indices[0], word_indices[-1]
                            labels[start_idx] = 'B'
                            for idx in range(start_idx + 1, end_idx + 1):
                                labels[idx] = 'I'
                        # for idx in word_indices:
                        #     if sent.words[idx].start_char == begin:
                        #         labels[idx] = 'B'
                        #     elif sent.words[idx].start_char > begin and sent.words[idx].end_char < end:
                        #         labels[idx] = 'I'

                    sentence_info.append({
                        'sentence_id': sent.index,
                        'words': tokens,
                        'labels': labels
                    })

                    # Print the result for each sentence
                    print('Sentence:', ' '.join(tokens))
                    print('Labels:', ' '.join(labels))
                    print()

                note_info['sentence_info'] = sentence_info

            extracted_texts.append(note_info)  # Append note information to the global variable

#

In [15]:
def find_word_indices(words, begin, end):
    # Create an empty list to store the indices of words found within the given range
    word_indices = []

    # Iterate through each token in the list of tokens
    for idx, word in enumerate(words):
        # Check if the start character of the token matches the beginning of the range
        # or if the token spans the beginning of the range
        if word.start_char == begin or (word.start_char < begin and word.end_char > begin):
            # If it matches, add the index of the token to the list of word indices
            word_indices.append(idx)
        # Check if the end character of the token matches the end of the range
        # or if the token spans the end of the range
        if word.end_char == end or (word.start_char < end and word.end_char > end):
            # If it matches, add the index of the token to the list of word indices
            word_indices.append(idx)
            break

    # Return the list of word indices found within the given range
    return word_indices

In [16]:
def generate_labels(words, annotations, start_token_idx, end_token_idx):
    labels = ['O'] * len(words)

    for annotation in annotations:
        code = annotation['code']

        # # Update labels for the matched words
        # if start_token_idx is not None and end_token_idx is not None:
        #     labels[start_token_idx] = f'B-{code}'
        #     for idx in range(start_token_idx + 1, end_token_idx + 1):
        #         labels[idx] = f'I-{code}'

    return labels

In [17]:
# Function to search for JSON files in a given folder and its subfolders

def search_files(folder_path):
    # Set to store unique hadm_id values
    hadm_id_set = set()

    # Recursively search for JSON files in the folder and its subfolders
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.endswith('.json'):
                # Construct the full path to the JSON file
                json_file_path = os.path.join(root, filename)
                print("Processing:", json_file_path)
                # Call extract_info_from_json function to extract information from the JSON file
                extract_info_from_json(json_file_path, hadm_id_set)
                # Print a separator after processing each file
                print("=" * 50)

    # Print the count of unique hadm_id values
    print("Total unique hadm_id count:", len(hadm_id_set))



In [18]:
# Specify the path to the main folder containing subfolders with JSON files
main_folder_path = '/content/drive/MyDrive/#medical coding/testing'

# Call the search_files function to start searching for JSON files in the main folder and its subfolders
search_files(main_folder_path)

Streaming output truncated to the last 5000 lines.

Sentence: Brief Hospital Course : # Lower GI Bleed : Admitted with a Hct of 29 from baseline 43 and orthostatic by vital signs .
Labels: O O O O O O O O O O O O O O O O O O O O O O O O

Sentence: She was aggressively volume resuscitated with 5 L of crystalloid and transfused 2 units of PRBCs after continuning to pass dilute blood with a Golytely prep , which was then held the first night of the hospitalization after completing half of the prep .
Labels: O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O

Sentence: On hospital day 2 , she underwent colonoscopy , which was remarkable for bleeding in the cecum , the site of 2 of her polypectomies 9 days prior to admission ; 4 clips were placed with adequate hemostasis .
Labels: O O O O O O O O O O O O O B I I I I I I I I I I I O O O O O O O O O O O O O O

Sentence: Her volume and hematocrit subsequently remained stable .
Labels: O O O O O O O O

Sente

In [19]:
import pandas as pd

# Function to create a DataFrame from the extracted sentence information
def create_dataframe(extracted_texts):
    data = []

    for note_info in extracted_texts:
        if 'sentence_info' in note_info:
            for sentence_info in note_info['sentence_info']:
                words = sentence_info['words']
                labels = sentence_info['labels']

                for word, label in zip(words, labels):
                    data.append({
                        'hadm_id': note_info['hadm_id'],
                        'note_id': note_info['note_id'],
                        'sentence_id': sentence_info['sentence_id'],
                        'word': word,
                        'label': label
                    })

    df = pd.DataFrame(data)
    return df

df = create_dataframe(extracted_texts)
print(df)

       hadm_id  note_id  sentence_id           word label
0       101525   566615            0   Demographics     O
1       101525   566615            0            Day     O
2       101525   566615            0             of     O
3       101525   566615            0     intubation     O
4       101525   566615            0              :     O
...        ...      ...          ...            ...   ...
40837   100463    55049          107            **]     O
40838   100463    55049          108      Completed     O
40839   100463    55049          108             by     O
40840   100463    55049          108  :[**2109-7-31     O
40841   100463    55049          108            **]     O

[40842 rows x 5 columns]


In [20]:
label_counts = df['label'].value_counts()
print(label_counts)

O    39926
I      582
B      334
Name: label, dtype: int64


In [21]:
token_df= df.drop(['hadm_id', 'note_id',], axis=1)

In [22]:
token_df

,sentence_id,word,label
0,0,Demographics,O
1,0,Day,O
2,0,of,O
3,0,intubation,O
4,0,:,O
...,...,...,...
40837,107,**],O
40838,108,Completed,O
40839,108,by,O
40840,108,:[**2109-7-31,O


In [23]:
token_df.rename(columns = {'word':'words', 'label':'labels'}, inplace = True)

In [24]:
token_df.head()

,sentence_id,words,labels
0,0,Demographics,O
1,0,Day,O
2,0,of,O
3,0,intubation,O
4,0,:,O


In [25]:
x = token_df[['sentence_id', 'words']]
y = token_df['labels']

SPLITTING DATA

In [26]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)

In [27]:
#build the training and test data
train_data = pd.DataFrame({'sentence_id': x_train['sentence_id'], 'words': x_train['words'], 'labels': y_train})
test_data = pd.DataFrame({'sentence_id': x_test['sentence_id'], 'words': x_test['words'], 'labels': y_test})

In [28]:
train_data

,sentence_id,words,labels
19837,12,:,O
30180,89,adjacent,O
13818,79,nadolol,O
18732,35,any,O
22160,22,Comments,O
...,...,...,...
31932,195,pm,O
19903,13,5,O
29413,21,has,O
10899,28,",",O


MODEL TRAINING

BERT

In [30]:
pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.5/263.5 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 

In [31]:
from simpletransformers.ner import NERModel, NERArgs

In [40]:
label = token_df['labels'].unique().tolist()
label

['O', 'B', 'I']

In [48]:
args = NERArgs()
args.num_train_epochs = 50
args.learning_rate = 1e-4
args.overwrite_output_dir = True
args.train_batch_size = 32
args.eval_batch_size = 32

In [49]:
model = NERModel('bert', 'bert-base-cased', labels=label, args=args, use_cuda = False)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [50]:
from sklearn.metrics import accuracy_score

model.train_model(train_data, eval_data = test_data, acc = accuracy_score)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Running Epoch 1 of 50:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 2 of 50:   0%|          | 0/7 [00:00<?, ?it/s]

...

Running Epoch 47 of 50:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 48 of 50:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 49 of 50:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 50 of 50:   0%|          | 0/7 [00:00<?, ?it/s]

(350, 0.03464245296427959)

In [51]:
result, model_outputs, preds_list = model.eval_model(test_data)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/6 [00:00<?, ?it/s]

In [52]:
result

{'eval_loss': 0.09497480542844035,
 'precision': 0.5581395348837209,
 'recall': 0.22018348623853212,
 'f1_score': 0.31578947368421056}

In [53]:
prediction, model_output = model.predict(['There is also intraventricular extension into the ipsilateral and contralateral lateral ventricles.'])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [54]:
prediction

[[{'There': 'O'},
  {'is': 'O'},
  {'also': 'O'},
  {'intraventricular': 'O'},
  {'extension': 'O'},
  {'into': 'O'},
  {'the': 'O'},
  {'ipsilateral': 'O'},
  {'and': 'O'},
  {'contralateral': 'O'},
  {'lateral': 'O'},
  {'ventricles.': 'O'}]]

BioBERT

In [55]:
bio_label = token_df['labels'].unique().tolist()
bio_label

['O', 'B', 'I']

In [56]:
# Initialize NER model arguments
args = NERArgs()
args.num_train_epochs = 10  # Number of training epochs
args.learning_rate = 2e-5   # Learning rate
args.overwrite_output_dir = True
args.train_batch_size = 32
args.eval_batch_size = 32

In [59]:
# Create the NER model with BioBERT
model = NERModel('bert', 'dmis-lab/biobert-base-cased-v1.2', args=args, labels=bio_label, use_cuda=False)


pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

In [60]:
# Train the model
model.train_model(train_data)

# Evaluate the model on test data
bio_result, model_outputs, preds_list = model.eval_model(test_data)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 2 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

...

Running Epoch 10 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [61]:
bio_result

{'eval_loss': 0.05618969723582268,
 'precision': 0.0,
 'recall': 0.0,
 'f1_score': 0.0}

In [62]:

predictions, raw_outputs = model.predict(["T[**Name2 (NI) **] was extubated on [**10-24**] without"])
print(predictions)


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'T[**Name2': 'O'}, {'(NI)': 'O'}, {'**]': 'O'}, {'was': 'O'}, {'extubated': 'O'}, {'on': 'O'}, {'[**10-24**]': 'O'}, {'without': 'O'}]]
